In [1]:
from helper import HelperTest
import unittest

suite = unittest.TestLoader().loadTestsFromModule(HelperTest())
unittest.TextTestRunner().run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [2]:
from tx import TxTest
import unittest

suite = unittest.TestLoader().loadTestsFromModule(TxTest())
unittest.TextTestRunner().run(suite)

.............
----------------------------------------------------------------------
Ran 13 tests in 3.610s

OK


<unittest.runner.TextTestResult run=13 errors=0 failures=0>

In [3]:
version = 1
length = 4
print(version.to_bytes(length, byteorder='little'))

b'\x01\x00\x00\x00'


In [4]:
from helper import double_sha256
from binascii import hexlify, unhexlify

modified_tx = '0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000'
h = double_sha256(unhexlify(modified_tx))
print(hexlify(h))
z = int(hexlify(h), 16)

b'27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6'


In [5]:
from binascii import unhexlify
from ecc import S256Point, Signature

sec = unhexlify('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
point = S256Point.from_sec(sec)

z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
r = 0xed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f
s = 0x7a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed

sig = Signature(r, s)
point.verify(z, sig)

True

In [6]:
from binascii import hexlify, unhexlify
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script
from tx import TxIn, TxOut, Tx

tx_ins = []
prev_tx = unhexlify('0025bc3c0fa8b7eb55b9437fdbd016870d18e0df0ace7bc9864efc38414147c8')
prev_index = 0
script_sig = b'\x00'
sequence = 0xffffffff
tx_ins.append(TxIn(prev_tx=prev_tx, prev_index=prev_index, script_sig=script_sig, sequence=sequence))
tx_outs = []
h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
tx_outs.append(TxOut(amount=int(0.99*100000000), script_pubkey=p2pkh_script(h160)))
h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
tx_outs.append(TxOut(amount=int(0.1*100000000), script_pubkey=p2pkh_script(h160)))
tx = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)
z = tx.hash_to_sign(0)
pk = PrivateKey(secret=8675309)
der = pk.sign(z).der()
sig = der + b'\x01'
sec = unhexlify(pk.point.sec())
script_sig = bytes([len(sig)]) + sig + bytes([len(sec)]) + sec
script_sig = bytes([len(script_sig)]) + script_sig
tx.tx_ins[0].script_sig = script_sig
print(hexlify(tx.serialize()))


b'0100000001c847414138fc4e86c97bce0adfe0180d8716d0db7f43b955ebb7a80f3cbc2500000000006c6b48304502210095aecf3775fa3ffa2a41eedd6271a9ccfcf295a6c7278ef503d2926c6b4794af022033ffed93827501eff7cd323ae81c432a642112cc529a9d9921679f8bbf0333fe012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff02c09ee605000000001976a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f88ac80969800000000001976a914507b27411ccf7f16f10297de6cef3f291623eddf88ac00000000'
